# Parameters

In [ ]:
TEST_RATIO = 1

In [ ]:
SAMPLE_INTERVAL = 60
HAPPY_COOLING_TIME = 30
SAD_COOLING_TIME = 60 * 6

NOTIFY_RATE = 60 * 8
TRADE_RATE = 10 << 10
BACKUP_RATE = 1

TRADE_UNIT = 0.0001
INIT_BUY_JPY = 5000
INIT_SELL_JPY = 5000
MAX_TRADED_JPY = 350000

TRADABLE_UNIT_CC_GAIN_JPY = 25000 # TODO: self-adaptive by trend analysis
TRADABLE_UNIT_CC_LOSS_JPY = 125000

MIN_UNIT_CC_INIT_TRADE_JPY = 10000
MAX_UNIT_CC_INIT_TRADE_JPY = MIN_UNIT_CC_INIT_TRADE_JPY << 10
MAX_UNIT_CC_LOSS_JPY = MAX_UNIT_CC_INIT_TRADE_JPY

MAX_GAIN_JPY = MAX_TRADED_JPY << 10 # useless
MAX_LOSS_JPY = MAX_TRADED_JPY << 10 # useless
MAX_TOTAL_LOST_JPY = MAX_TRADED_JPY << 10

# Status

In [ ]:
%%bash
pip3 install pyyaml

In [ ]:
import os
import yaml


class Status(object):
    PATH = os.getenv('ROBOT_PATH') or '.'
    STATUS_FILE = f'{PATH}/robot-status.yaml'
    TMPFILE = f'{PATH}/_tmp'

    def __init__(self, robot_name):
        self.robot_name = robot_name
        self.trade_unit = TRADE_UNIT

        self.now_buy_fiat_price = None
        self.now_sell_fiat_price = None
        self.please_buy_unit_amount = 0
        self.please_sell_unit_amount = 0
        self.cooling_time = HAPPY_COOLING_TIME

        self.sample_number = 0
        self.trade_count = 0

        self.buy_count = 0
        self.clear_bought_status()

        self.sell_count = 0
        self.clear_sold_status()

        self.total_gained_fiat_money = 0

    def get_usage(self):
        return self.used_fiat_money / MAX_TRADED_JPY

    def get_robot_title(self):
        robot_status_description = f'{self.get_usage() * 100:.2f}%'
        if self.bought_average_fiat_price is not None:
            robot_status_description = f'{self.bought_average_fiat_price:.2f} / {robot_status_description}'
        return f'{self.robot_name} ({robot_status_description}) at {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'

    def get_gain_fiat_money(self):
        return self.now_sell_fiat_price * self.bought_amount - self.used_fiat_money

    def get_total_gain_fiat_money(self):
        return self.total_gained_fiat_money + self.get_gain_fiat_money()

    def __str__(self):
        sell_fiat_price_description = f'{self.now_sell_fiat_price:.2f}'
        if self.bought_average_fiat_price is not None:
            diff_fiat_price = self.now_sell_fiat_price - self.bought_average_fiat_price
            sell_fiat_price_description = f'{diff_fiat_price:.2f} = {self.now_sell_fiat_price:.2f} - {self.bought_average_fiat_price:.2f}'
        return (
            f'GAIN {self.get_total_gain_fiat_money():+.2f} = {self.total_gained_fiat_money:+.2f} {self.get_gain_fiat_money():+.2f} JPY in total by {self.get_robot_title()}\n'
            f'SELL {self.bought_amount:.4f} ETH for {sell_fiat_price_description} JPY/ETH. BUY for {self.now_buy_fiat_price:.2f} JPY/ETH.\n'
        )

    def estimate_gained_fiat_money(self, sell_unit_amount, sell_fiat_price=None):
        if sell_unit_amount <= 0:
            return
        if sell_unit_amount > self.bought_unit_amount:
            return
        if self.bought_average_fiat_price is None:
            return
        if sell_fiat_price is None:
            sell_fiat_price = self.now_sell_fiat_price
        return (sell_fiat_price - self.bought_average_fiat_price) * (self.trade_unit * sell_unit_amount)

    def read(self):
        if not os.path.exists(type(self).STATUS_FILE):
            return
        with open(type(self).STATUS_FILE, 'r') as f:
            self.__dict__.update(
                yaml.load(f, Loader=yaml.FullLoader)
            )

    def write(self):
        with open(type(self).TMPFILE, 'w') as f:
            yaml.dump(self.__dict__, f, sort_keys=False)
        os.replace(type(self).TMPFILE, type(self).STATUS_FILE)

    def clear_bought_status(self):
        self.bought_unit_amount = 0
        self.used_fiat_money = 0
        self.bought_amount = 0
        self.bought_average_fiat_price = None # MAX_UNIT_CC_INIT_TRADE_JPY

    def clear_sold_status(self):
        self.sold_unit_amount = 0
        self.got_fiat_money = 0
        self.sold_amount = 0
        self.sold_average_fiat_price = None # MAX_UNIT_CC_INIT_TRADE_JPY

    def trim_bought_status(self):
        if self.bought_unit_amount > 0:
            self.bought_amount = self.trade_unit * self.bought_unit_amount
            self.bought_average_fiat_price = self.used_fiat_money / self.bought_amount
        else:
            self.clear_bought_status()

    def trim_sold_status(self):
        if self.sold_unit_amount > 0:
            self.sold_amount = self.trade_unit * self.sold_unit_amount
            self.sold_average_fiat_price = self.got_fiat_money / self.sold_amount
        else:
            self.clear_sold_status()

    def update_bought_status(self, **trade):
        buy_fiat_price = trade.get('fiat_price', self.now_buy_fiat_price)
        buy_unit_amount = trade.get('unit_amount', 0)
        self.bought_unit_amount += buy_unit_amount
        self.used_fiat_money += buy_fiat_price * (self.trade_unit * buy_unit_amount)
        self.trim_bought_status()

    def update_sold_status(self, **trade):
        sell_fiat_price = trade.get('fiat_price', self.now_sell_fiat_price)
        sell_unit_amount = trade.get('unit_amount', 0)
        self.sold_unit_amount += sell_unit_amount
        self.got_fiat_money += sell_fiat_price * (self.trade_unit * sell_unit_amount)
        self.trim_sold_status()

    def trim_trade_status(self):
        self.trim_bought_status()
        self.trim_sold_status()

    def update(self, status=None):
        if isinstance(status, type(self)):
            self.__dict__.update(status.__dict__)
        if isinstance(status, dict):
            self.__dict__.update(status)
        self.trim_trade_status()


status = Status('Long-Term Robot')
status.read()

# BookKeeper

In [ ]:
import os
WEBHOOK_URL = os.environ['SLACK_WEBHOOK_URL']
GOOD_CHANNEL_ID = os.environ['SLACK_GOOD_CHANNEL_ID']
DANGER_CHANNEL_ID = os.environ['SLACK_DANGER_CHANNEL_ID']

In [ ]:
def send_slack(message, text, color='good'):
    if color == 'good':
        channel_id = GOOD_CHANNEL_ID
        message = f':grinning: {message}'
    if color == 'danger':
        channel_id = DANGER_CHANNEL_ID
        message = f':slightly_frowning_face: {message}'

    import requests
    requests.post(
        WEBHOOK_URL,
        json={
            'channel': channel_id,
            'username': 'ETH Notifications',
            'icon_url': 'https://ethereum.org/favicon-32x32.png',
            'text': message,
            'mrkdwn': True,
            'attachments': [{
                'title': '→ Trading Operations',
                'title_link': 'https://jhub.name/user/sheng_wei/lab',
                'text': text,
                'color': color,
            }]
        }
    )

In [ ]:
%%bash
pip3 install git+https://github.com/seii-saintway/python-bitbankcc.git@fix/too-many-open-files

In [ ]:
import copy
from datetime import datetime
from decimal import Decimal
import os

import python_bitbankcc


class Portfolio(object):
    def __init__(self, portfolio):
        if isinstance(portfolio, type(self)):
            self.__dict__.update(portfolio.__dict__)
        if isinstance(portfolio, dict):
            self.__dict__.update(portfolio)
        if isinstance(portfolio, list):
            self.__dict__.update(dict(portfolio))


class BookKeeper(object):
    def __init__(self, status):
        self.pub = python_bitbankcc.public()
        self.prv = python_bitbankcc.private(
            os.environ['BITBANK_API_KEY'],
            os.environ['BITBANK_API_SECRET']
        )

        self.status = status
        self.portfolio = self.get_portfolio()
        self.last_transaction = None

    def __enter__(self):
        self.fsh = open('sample-history.txt', 'a', 1)
        self.fth = open('transaction-history.txt', 'a', 1)
        return self
  
    def __exit__(self):
        self.fsh.close()
        self.fth.close()

    def get_price(self):
        now_ticker = self.pub.get_ticker('eth_jpy')
        return {
            'now_sell_fiat_price': int(now_ticker.get('buy', '0')) * (1 - 0.0012),
            'now_buy_fiat_price': int(now_ticker.get('sell', '1')) * (1 + 0.0012)
        }

    def get_portfolio(self):
        return Portfolio([
            (asset.get('asset', ''), Decimal(asset.get('onhand_amount', '0'))) for asset in self.prv.get_asset().get('assets', [])
        ])

    def estimate_status(self):
        old_portfolio = self.portfolio
        new_portfolio = self.get_portfolio()
        self.portfolio = new_portfolio

        old_fiat_money = old_portfolio.jpy
        old_amount = old_portfolio.eth
        new_fiat_money = new_portfolio.jpy
        new_amount = new_portfolio.eth

        if old_fiat_money > new_fiat_money and old_amount < new_amount:
            amount = new_amount - old_amount
            fiat_price = (old_fiat_money - new_fiat_money) / amount

            if self.status.now_buy_fiat_price is None:
                return self.status
            now_buy_fiat_price = Decimal(self.status.now_buy_fiat_price)
            if abs(fiat_price - now_buy_fiat_price) / now_buy_fiat_price > Decimal(50/100):
                return self.status

            new_status = self.estimate_status_by_buying(
                int( amount / Decimal(repr(self.status.trade_unit)) ),
                float( fiat_price )
            )

            new_status.update(
                status = {
                    'sold_unit_amount': 0,
                    'please_buy_unit_amount': 0
                }
            )

            return new_status

        if old_fiat_money < new_fiat_money and old_amount > new_amount:
            amount = old_amount - new_amount
            fiat_price = (new_fiat_money - old_fiat_money) / amount

            if self.status.now_sell_fiat_price is None:
                return self.status
            now_sell_fiat_price = Decimal(self.status.now_sell_fiat_price)
            if abs(fiat_price - now_sell_fiat_price) / now_sell_fiat_price > Decimal(50/100):
                return self.status

            new_status = self.estimate_status_by_selling(
                int( amount / Decimal(repr(self.status.trade_unit)) ),
                float( fiat_price )
            )

            new_status.update(
                status = {
                    'please_buy_unit_amount': 0,
                    'please_sell_unit_amount': 0
                }
            )

            if new_status.bought_unit_amount == 0:
                new_status.update(
                    status = {
                        'trade_count': status.trade_count + 1,
                        'buy_count': 0,
                        'sell_count': 0
                    }
                )

            return new_status

        return self.status

    def buy_secretly(self, unit_amount, fiat_price=None):
        if unit_amount <= 0:
            return
        if fiat_price is None:
            fiat_price = self.status.now_buy_fiat_price

        amount = self.status.trade_unit * unit_amount
        self.prv.order(
            pair='eth_jpy', price=str(fiat_price), amount=str(amount), side='buy', order_type='market'
        )

    def buy(self, unit_amount, fiat_price=None):
        if unit_amount <= 0:
            return
        if fiat_price is None:
            fiat_price = self.status.now_buy_fiat_price

        self.buy_secretly(unit_amount, fiat_price)

        return self.estimate_status_by_buying(unit_amount, fiat_price)

    def estimate_status_by_buying(self, unit_amount, fiat_price=None):
        if unit_amount <= 0:
            return
        if fiat_price is None:
            fiat_price = self.status.now_buy_fiat_price

        amount = self.status.trade_unit * unit_amount

        old_status = self.status
        new_status = copy.deepcopy(old_status)

        new_status.buy_count += 1
        new_status.update_bought_status(fiat_price=fiat_price, unit_amount=unit_amount)

        old_amount = old_status.bought_amount
        old_percentage = old_status.get_usage() * 100
        new_amount = new_status.bought_amount
        new_percentage = new_status.get_usage() * 100

        self.last_transaction = (
            f'BOUGHT {amount:.4f} = -{old_amount:.4f} ({old_percentage:.2f}%) +{new_amount:.4f} ({new_percentage:.2f}%) ETH for {fiat_price:.2f} JPY/ETH:'
            f' NO.{new_status.trade_count + 1}-{new_status.buy_count} at {datetime.now()}\n'
        )

        self.fth.write(self.last_transaction)

        return new_status

    def sell_secretly(self, unit_amount, fiat_price=None):
        if unit_amount <= 0:
            return
        if fiat_price is None:
            fiat_price = self.status.now_sell_fiat_price

        amount = self.status.trade_unit * unit_amount
        self.prv.order(
            pair='eth_jpy', price=str(fiat_price), amount=str(amount), side='sell', order_type='market'
        )

    def sell(self, unit_amount, fiat_price=None):
        if unit_amount <= 0:
            return
        if fiat_price is None:
            fiat_price = self.status.now_sell_fiat_price

        self.sell_secretly(unit_amount, fiat_price)

        return self.estimate_status_by_selling(unit_amount, fiat_price)

    def estimate_status_by_selling(self, unit_amount, fiat_price=None):
        if unit_amount <= 0:
            return
        if fiat_price is None:
            fiat_price = self.status.now_sell_fiat_price

        amount = self.status.trade_unit * unit_amount

        old_status = self.status
        new_status = copy.deepcopy(old_status)

        new_status.sell_count += 1
        new_status.update_bought_status(fiat_price=new_status.bought_average_fiat_price, unit_amount=-unit_amount)
        new_status.update_sold_status(fiat_price=fiat_price, unit_amount=unit_amount)

        old_avg_price = old_status.bought_average_fiat_price
        diff_price = fiat_price - old_avg_price
        gained_fiat_money = old_status.estimate_gained_fiat_money(unit_amount, fiat_price)
        new_status.total_gained_fiat_money += gained_fiat_money

        old_amount = old_status.bought_amount
        old_percentage = old_status.get_usage() * 100
        new_amount = new_status.bought_amount
        new_percentage = new_status.get_usage() * 100

        self.last_transaction = (
            f'GAINED {gained_fiat_money:+.2f} JPY: GAINED {new_status.total_gained_fiat_money:+.2f} JPY in total at {datetime.now()}\n'
            f'SOLD {amount:.4f} = +{old_amount:.4f} ({old_percentage:.2f}%) -{new_amount:.4f} ({new_percentage:.2f}%) ETH'
            f' for {diff_price:.2f} = {fiat_price:.2f} - {old_avg_price:.2f} JPY/ETH: NO.{new_status.trade_count + 1}-{new_status.sell_count}\n'
        )

        self.fth.write(self.last_transaction)

        return new_status

# A Trader Stabilize Fiat Value of Crypto Currency

In [ ]:
%%bash
pip3 install scipy

In [ ]:
import math
from scipy import optimize as opt
from numpy.lib import scimath

def get_trade_unit_amount(now_trade_jpy, traded_unit_amount, traded_jpy, tradable_unit_cc_diff_jpy, init_jpy):
    if traded_unit_amount == 0:
        return 1
    avg_jpy = traded_jpy / (TRADE_UNIT * traded_unit_amount)
    diff_jpy = abs(now_trade_jpy - avg_jpy)
    min_diff_jpy = tradable_unit_cc_diff_jpy / scimath.log(MAX_TRADED_JPY / init_jpy)
    return math.ceil(
        traded_jpy * opt.fsolve(
            lambda c: [
                (
                    diff_jpy - min_diff_jpy * scimath.log(1 + c[0]) / (avg_jpy * c[0]) * (avg_jpy * c[0] + now_trade_jpy)
                ).real
            ],
            [-0.5]
        )[0] / now_trade_jpy / TRADE_UNIT
    )

## Start Trading

In [ ]:
send_slack(
    f'{status.robot_name} started\n{status.get_robot_title()}', 'Power by https://jhub.name/', 'good'
)

In [ ]:
import time
from decimal import Decimal
with BookKeeper(status) as bookkeeper:
#     from unittest.mock import Mock
#     bookkeeper.prv.order = Mock()
    while True:
        new_status = bookkeeper.estimate_status()
        if new_status is not status:
            bookkeeper.fth.write(f'{new_status}\n')
            new_status.write()

            send_slack(
                f'{bookkeeper.last_transaction}'
                f'BY {new_status.get_robot_title()}',
                'Power by https://jhub.name/', 'good' if new_status.total_gained_fiat_money > 0 else 'danger'
            )

            send_slack(
                f'{new_status} => Support level is {MIN_UNIT_CC_INIT_TRADE_JPY} JPY.',
                'Power by https://jhub.name/', 'good' if new_status.get_total_gain_fiat_money() > 0 else 'danger'
            )

            if new_status.bought_unit_amount == 0 and new_status.total_gained_fiat_money > MAX_GAIN_JPY << 2 and not isinstance(bookkeeper.prv, Mock):
                from unittest.mock import Mock
                bookkeeper.prv = Mock()
                send_slack(
                    f'{new_status.robot_name} is turned to Mock', 'Power by https://jhub.name/', 'good'
                )

            if new_status.total_gained_fiat_money < -MAX_TOTAL_LOST_JPY:
                break

            if new_status.total_gained_fiat_money == status.total_gained_fiat_money:
                time.sleep(SAMPLE_INTERVAL / TEST_RATIO)
            else:
                time.sleep(new_status.cooling_time/TEST_RATIO)
                status.cooling_time = HAPPY_COOLING_TIME

        status.update(new_status)
        status.update(bookkeeper.get_price())

        if status.sample_number % TRADE_RATE == 0 and status.bought_average_fiat_price is not None and status.now_sell_fiat_price > status.bought_average_fiat_price:
            status.please_sell_unit_amount = 1
            status.cooling_time = HAPPY_COOLING_TIME

        if status.sold_unit_amount == 0 and status.bought_average_fiat_price is not None and status.now_sell_fiat_price > status.bought_average_fiat_price:
            status.sample_number = 0
            status.please_sell_unit_amount = min(math.ceil(INIT_SELL_JPY / status.now_sell_fiat_price / status.trade_unit), status.bought_unit_amount)
            status.cooling_time = HAPPY_COOLING_TIME

        if status.sold_unit_amount > 0 and status.now_sell_fiat_price > status.sold_average_fiat_price:
            trade_unit_amount = min(
                get_trade_unit_amount(status.now_sell_fiat_price, status.sold_unit_amount, status.got_fiat_money, TRADABLE_UNIT_CC_GAIN_JPY, INIT_SELL_JPY),
                status.bought_unit_amount
            )
            if trade_unit_amount > 0:
                status.sample_number = 0
                status.please_sell_unit_amount = trade_unit_amount
                status.cooling_time = HAPPY_COOLING_TIME

        gained_fiat_money = status.estimate_gained_fiat_money(status.bought_unit_amount)
        if gained_fiat_money is not None and gained_fiat_money > MAX_GAIN_JPY:
            status.sample_number = 0
            status.please_sell_unit_amount = status.bought_unit_amount
            status.cooling_time = HAPPY_COOLING_TIME

        if gained_fiat_money is not None and gained_fiat_money < -MAX_LOSS_JPY:
            status.sample_number = 0
            status.please_sell_unit_amount = status.bought_unit_amount
            status.cooling_time = SAD_COOLING_TIME

        if status.bought_average_fiat_price is not None and status.now_sell_fiat_price < status.bought_average_fiat_price - MAX_UNIT_CC_LOSS_JPY and status.get_usage() > 90/100:
            status.sample_number = 0
            status.please_sell_unit_amount = status.bought_unit_amount
            status.cooling_time = SAD_COOLING_TIME

        if status.bought_unit_amount > 0 and status.now_sell_fiat_price < MIN_UNIT_CC_INIT_TRADE_JPY:
            status.please_sell_unit_amount = status.bought_unit_amount
            status.cooling_time = SAD_COOLING_TIME

        if status.please_sell_unit_amount > 0:
            bookkeeper.sell_secretly(status.please_sell_unit_amount)
            bookkeeper.fsh.write(f'{status}\n')
            continue

        if status.used_fiat_money < INIT_BUY_JPY and INIT_BUY_JPY - status.used_fiat_money <= (MAX_TRADED_JPY - status.used_fiat_money) * 0.75 \
                and status.now_buy_fiat_price > MIN_UNIT_CC_INIT_TRADE_JPY and status.now_buy_fiat_price < MAX_UNIT_CC_INIT_TRADE_JPY:
            status.sample_number = 0
            status.please_buy_unit_amount = math.ceil((INIT_BUY_JPY - status.used_fiat_money) / status.now_buy_fiat_price / status.trade_unit)

        if status.used_fiat_money >= INIT_BUY_JPY and status.bought_average_fiat_price is not None and status.now_buy_fiat_price < status.bought_average_fiat_price:
            trade_unit_amount = get_trade_unit_amount(status.now_buy_fiat_price, status.bought_unit_amount, status.used_fiat_money, TRADABLE_UNIT_CC_LOSS_JPY, INIT_BUY_JPY)
            while trade_unit_amount > 0 and status.now_buy_fiat_price * status.trade_unit * trade_unit_amount > (MAX_TRADED_JPY - status.used_fiat_money) * 0.75:
                trade_unit_amount >>= 1
            if trade_unit_amount > 0:
                status.sample_number = 0
                status.please_buy_unit_amount = trade_unit_amount

        if status.please_buy_unit_amount > 0:
            bookkeeper.buy_secretly(status.please_buy_unit_amount)
            bookkeeper.fsh.write(f'{status}\n')
            continue

        bookkeeper.fsh.write(f'{status}\n')

        if status.sample_number % TRADE_RATE == 0:
            bookkeeper.fth.write(f'{status}\n')

        if status.sample_number % BACKUP_RATE == 0:
            status.write()

        if status.sample_number % NOTIFY_RATE == 0:
            send_slack(
                f'{status} => Support level is {MIN_UNIT_CC_INIT_TRADE_JPY} JPY.',
                'Power by https://jhub.name/', 'good' if status.get_total_gain_fiat_money() > 0 else 'danger'
            )

        status.sample_number += 1
        time.sleep(SAMPLE_INTERVAL / TEST_RATIO)